In [29]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
from dash import no_update
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True, errors="ignore")

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Shared styles
BASE_CONTAINER_STYLE = {
    "maxWidth": "1100px",
    "margin": "0 auto",
    "fontFamily": "system-ui, -apple-system, Segoe UI, Roboto, sans-serif",
}

CARD_STYLE = {
    "padding": "8px 12px",
    "border": "1px solid #e6e6e6",
    "borderRadius": "12px",
    "boxShadow": "0 1px 8px rgba(0, 0, 0, 0.06)",
    "background": "white",
}

# Table zebra striping and active state
BASE_TABLE_STYLES = [
    {"if": {"row_index": "odd"}, "backgroundColor": "#f9f9f9"},
    {"if": {"row_index": "even"}, "backgroundColor": "white"},
    {"if": {"state": "active"}, "backgroundColor": "#d2f3ff", "border": "1px solid #007aac"},
]

BUTTON_PILL = {
    'display': 'inline-block',
    'padding': '10px 20px',
    'margin': '8px',
    'border': '2px solid #ccc',
    'borderRadius': '#f9f9f9',
    'cursor': 'pointer',
    'fontSize': '18px'
}

#FIX ME Add in Grazioso Salvare’s logo
image_filename = "Grazioso Salvare Logo.png"

if os.path.exists(image_filename):
    with open(image_filename, "rb") as img_file:
        encoded_image = base64.b64encode(img_file.read()).decode()
else:
    print("Logo file not found:", image_filename)
    encoded_image = None

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div(
#    html.Div(id='hidden-div', style={'display':'none'}),
    style=BASE_CONTAINER_STYLE,
    children=[
        html.Center([
            html.Img(
                src=f"data:image/png;base64,{encoded_image}" if encoded_image else "",
                style={'height': '270px', 'marginBottom': '30px'}
            ),
            html.H1('Grazioso Salvare Rescue Breeds Dashboard', style={'margin': '0'}),
            html.H3('Dustin Davis • Project Two', style={'marginTop': '5px', 'marginBottom': '20px'})
        ]),
        
        html.Hr(),
        
        html.Div(
            style={"display": "flex", "gap": "12px", "justifyContent": "center", "alignItems": "center", "flexWrap": "wrap"},
            children=[        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
                dcc.RadioItems(
                    id='filter-type',
                    options=[
                        {'label': 'Water Rescue', 'value': 'water'},
                        {'label': 'Mountain or Wilderness', 'value': 'mountain'},
                        {'label': 'Disaster or Individual Tracking', 'value': 'disaster'}
                    ],
                    value='all',
                    inline=True,
                    labelStyle={
                        'display': 'inline-block',
                        'padding': '10px 20px',
                        'margin': '8px',
                        'border': '2px solid #ccc',
                        'borderRadius': '8px',
                        'backgroundColor': '#f9f9f9', 
                        'cursor': 'pointer',
                        'fontSize': '18px'
                    },
                    inputStyle={'marginRight': '8px', 'cursor': 'pointer'}
                ),
                html.Button("Reset Filters", id="btn-reset", n_clicks=0, style={"height": "40px"}),
                html.Button("Export CSV", id="btn-export", n_clicks=0, style={"height": "40px"}),
                dcc.Download(id="download-csv"),
            ],
        ),
        
        html.Hr(),
        
        html.Div(
            style=CARD_STYLE | {"marginBottom": "18px"},
            children=[
                html.Div(
                    id='map-id',
                    style={'width': '100%', 'margin': '0 auto'}
                )
            ]
        ),
        
        html.Hr(),
        
        html.Div(
            style={'display': 'flex', 'gap': '16px', 'alignItems': 'stretch', 'flexWrap': 'nowrap'},
            children=[
                html.Div(
                    style=CARD_STYLE | {'flex': '1 1 62%', 'minWidth': '380px'},
                    children=[
                        dash_table.DataTable(
                            id='datatable-id',
                            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                            data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                            page_size=10,
                            filter_action='native',
                            sort_action='native',
                            sort_mode='multi',
                            column_selectable='single',
                            row_selectable='single',
                            selected_rows=[0],
                            style_table={'overflowX': 'auto', 'width': '100%', 'margin': '0'},
                            style_cell={'textAlign': 'left', 'minWidth': '120px', 'whiteSpace': 'normal'},
                            style_header={'fontWeight': 'bold', 'backgroundColor': '#e9ecef'},
                            style_data_conditional=BASE_TABLE_STYLES,
                        )
                    ]
                ),
                        
                html.Div(
                    style=CARD_STYLE | {'flex': '1 1 38%', 'minWidth': '280px'},
                    children=[
                        html.Div(id='graph-id')
                    ]
                ),
            ]
        ),
        
        html.Br(),
    ]
)
    
#############################################
# Interaction Between Components / Controller
#############################################

# Helper to build Mongo queries for each rescue type
def _regex_breeds(breeds):
    escaped = [b.replace("(", r"\(").replace(")", r"\)") for b in breeds]
    pattern = "(" + "|".join(escaped) + ")"
    return {"$regex": pattern, "$options": "i"}
                              
def _query_for(rescue_type: str):
    base = {
        "animal_type": "Dog",
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},}
                              
    if rescue_type == 'water':
        breeds = ["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"]
        return base | {"breed": _regex_breeds(breeds)}
                              
    if rescue_type == 'mountain':
        breeds = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]
        return base | {"breed": _regex_breeds(breeds)}
                              
    if rescue_type == 'disaster':
        breeds = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
        return base | {"breed": _regex_breeds(breeds)}
                              
    return {}

    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
## FIX ME Add code to filter interactive data table with MongoDB queries
# Execute the matching Mongo query via the CRUD module and return rows to the table
    query = _query_for(filter_type)
    dff = pd.DataFrame.from_records(db.read(query))
    if not dff.empty and '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True, errors="ignore")
    if dff.empty:
        return []
    return dff.to_dict('records')
#
#
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame(viewData) if viewData else pd.DataFrame()
    
    if dff.empty or 'breed' not in dff.columns:
        empty_fig = px.bar(title='Top Breeds (current view) ')
        empty_fig.update_layout(
            annotations-[dict(text="No data to display", x=0.5, y=0.5, showarrow=False)],
            xaxis_title='Count', yaxis_title=''
        )
        reutn [dcc.Graph(figure=empty_fig)]
        
    counts = dff['breed'].value_counts().reset_index()
    counts.columns = ['breed', 'count']
    counts = counts.head(15)
    counts['count'] = pd.to_numeric(counts['count'], errors='coerce').fillna(0).astype(int)
    fig = px.bar(counts, x='count', y='breed', orientation='h', title='Top Breeds (current view)', text='count')
    fig.update_traces(textposition='outside', cliponaxis=False)
    xmax = int(counts['count'].max()) if len(counts) else 0
    fig.update_xaxes(range=[0, max(1, xmax * 1.15)])
    fig.update_layout(margin=dict(l=20, r=20, t=40, b=20), xaxis_title='Count', yaxis_title='', uniformtext_minsize=12, uniformtext_mode='hide')
    return [dcc.Graph(figure=fig)]

    #return [
    #    dcc.Graph(            
    #        figure = px.pie(df, names='breed', title='Preferred Animals')
    #    )    
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [
        Input('datatable-id', 'selected_columns'),
        Input('datatable-id', 'derived_virtual_selected_rows'),
    ]
)
def update_styles(select_columns, selected_rows):
    styles = list(BASE_TABLE_STYLES)
    
    # Highlight selected row (blue)
    if selected_rows:
        styles.append(
            {"if": {"row_index": selected_rows[0]},
             "backgroundColor": "#d2f3ff",
             "border": "1px solid #007aac"}
        )
        
    return styles

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    center_lat, center_lon = 30.75, -97.48    
    dff = pd.DataFrame(viewData) if viewData else pd.DataFrame()
    
    if dff.empty:
        return [dl.Map(style={'width': '100%', 'height': '500px'},
                       center=[center_lat, center_lon], zoom=10,
                       children=[dl.TileLayer(id="base-layer-id")])]
    # Because we only allow single row selection, the list can be converted to a row index here
    row = 0 if not index else index[0]
    
    # Prefer named columns; fall back to pisitional indices used in starter
    lat_col = 'location_lat' if 'location_lat' in dff.columns else dff.columns[13]
    lon_col = 'location_long' if 'location_long' in dff.columns else dff.columns[14]
    breed_col = 'breed' if 'breed' in dff.columns else dff.columns[4]
    name_col = 'name' if 'name' in dff.columns else dff.columns[9]
    
    lat = pd.to_numeric(dff.iloc[row][lat_col], errors='coerce')
    lon = pd.to_numeric(dff.iloc[row][lon_col], errors='coerce')
    if pd.isna(lat) or pd.isna(lon):
        lat, lon = center_lat, center_lon
    
    breed = str(dff.iloc[row][breed_col])
    name = str(dff.iloc[row][name_col])
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, 
               center=[float(lat), float(lon)], zoom=12, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[float(lat), float(lon)], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(name)])
                ])
            ])
        ]
                              
@app.callback(
    Output("download-csv", "data"),
    Input("btn-export", "n_clicks"),
    State("datatable-id", "derived_virtual_data"),
    State("datatable-id", "columns"),
    prevent_initial_call=True
)
def export_table_csv(n_clicks, view_data, columns):
    dff = pd.DataFrame(view_data) if view_data else pd.DataFrame()
    if dff.empty:
        return no_update
    ordered_cols = [c["id"] for c in (columns or []) if c["id"] in dff.columns]
    if ordered_cols:
        dff = dff[ordered_cols]
    return dcc.send_data_frame(dff.to_csv, "aac_export.csv", index=False)

@app.callback(
    Output('filter-type', 'value'),
    Output('datatable-id', 'filter_query'),
    Output('datatable-id', 'sort_by'),
    Output('datatable-id', 'page_current'),
    Output('datatable-id', 'selected_rows'),
    Input('btn-reset', 'n_clicks'),
    prevent_initial_call=True
)
def reset_all(n):
    return 'all', "", [], 0, [0]
# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on https://gallopslow-ridercartoon-3000.codio.io/proxy/8050/
